In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm 
import seaborn as sns
import math

from scipy.stats import gaussian_kde
from scipy.signal import argrelextrema

from scipy.sparse import issparse

import gc


# set figure size for better visibility
sc.settings.set_figure_params(dpi=100, frameon=False)
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)

## NORMALIZATION

Using Pearson residuals (see paper https://doi.org/10.1186/s13059-019-1874-1)

Uncouple poisson noise from technical issues from biological heterogeneity: handles the "depth" scaling and variance stabilization.

Notes:

1. We do NOT filter genes yet. The method needs most genes to fit the model.
2. We perform this on the WHOLE dataset (all plates/conditions mixed).
3. Might be needed to still need some extra correction for batch effects (Harmony / scVI). We will see this with the UMAP and mESC control.

In [ ]:
# --- LOAD DATA ---
print("Loading data...")
adata = sc.read_h5ad("../data/data_filtered_raw.h5ad")

# --- DATA TYPE DIAGNOSTICS (Sanity Check) ---
# We verify if the matrix contains integers (counts) or floats (already normalized?)

if issparse(adata.X):
    data_sample = adata.X.data[:10] # Look at first 10 non-zero elements
else:
    data_sample = adata.X[:10, :10].flatten()

print("--- MATRIX INSPECTION ---")
print(f"Data Sample: {data_sample}")
print(f"Data Type: {adata.X.dtype}")

# CHECK 1: Are they already normalized? (Small numbers < 1)
if np.any((data_sample > 0) & (data_sample < 1)):
    print("CRITICAL ERROR: Data appears to be already normalized (values < 1 found).")
    print("STOP: Pearson Residuals require RAW INTEGER COUNTS.")
    # Stop execution here if this happens! You need to reload raw data.

# CHECK 2: Are they floats looking like integers? (e.g. 5.0, 12.0)
# We calculate the sum of fractional parts. Should be 0.
if issparse(adata.X):
    fractional_part = np.sum(adata.X.data % 1)
else:
    fractional_part = np.sum(adata.X % 1)

if fractional_part == 0:
    print("Physics Check Passed: Data represents discrete quanta (Integers).")
    
    # Force casting to integer to satisfy the algorithm's strict requirements
    # This changes 5.0 to 5, silencing the warning.
    if adata.X.dtype != int:
        print("Casting matrix from Float to Int...")
        adata.X = adata.X.astype(int)
else:
    print("WARNING: Data has non-zero fractional parts but > 1.")
    print("Did you apply some scaling? Pearson Residuals might be inaccurate.")
    # Usually we round to nearest integer if we trust the raw nature
    adata.X = np.rint(adata.X).astype(int)

# --- SAVE RAW STATE (Backup) ---
# Before normalizing, we MUST save the raw counts.
# 'counts' layer is the standard slot for this.
adata.layers["counts"] = adata.X.copy()
print("Raw counts saved to adata.layers['counts'].")

# --- NORMALIZATION (Hafemeister & Satija / Pearson Residuals) ---
print("\n--- RUNNING NORMALIZATION ---")
# This creates the GLM model and calculates residuals
sc.experimental.pp.normalize_pearson_residuals(
    adata, 
    theta=100,      # Overdispersion parameter (100 is standard for UMI data)
    clip=True,      # Clips extreme outliers (√N) to stabilize variance
    check_values=True,
    layer="counts"  # Explicitly tell it to use the raw counts layer we just saved
)

print("Normalization Complete.")
print("adata.X now contains Pearson Residuals (Gaussian-like distribution).")
print(f"Range of new values: {adata.X.min():.2f} to {adata.X.max():.2f}")

Loading data...
--- MATRIX INSPECTION ---
Data Sample: [1 0 3 0 1 3 1 0 1 1]
Data Type: int64
Physics Check Passed: Data represents discrete quanta (Integers).
Raw counts saved to adata.layers['counts'].

--- RUNNING NORMALIZATION ---
computing analytic Pearson residuals on counts


MemoryError: Unable to allocate 6.02 GiB for an array with shape (3311, 244105) and data type float64

## PCA / UMAP

INTERPRETATION:

1. In the 'plate' coloring, mESC cluster is a mix of colors => ok
2. Distinct islands of "Plate 1 mESC" and "Plate 2 mESC" => batch correction needed (e.g., Harmony, BBKNN, scVI).

In [ ]:
# --- DIMENSIONALITY REDUCTION ---

# PCA (Principal Component Analysis)
sc.tl.pca(adata, n_comps=30, svd_solver='arpack')

# Calculate Neighbors (Graph construction)
# k-NN graph in PCA space. Needed for UMAP/Clustering.
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30)

# UMAP (Uniform Manifold Approximation and Projection)
sc.tl.umap(adata)

# --- BATCH EFFECT VISUALIZATION ---
# => THE "mESC" TEST
#
# Coloring by:
# - Sample Type: To see if mESC cluster together away from Ectoderm
# - Plate: To see if mESCs from Plate 1 overlap with mESCs from Plate 2

sc.pl.umap(
    adata, 
    color=['sample_type', 'plate', 'experiment'], 
    wspace=0.3
)